In [2]:
import tensorflow as tf

2025-02-02 00:48:25.517591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738457305.529199    1632 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738457305.532678    1632 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 00:48:25.544769: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Get the list of all logical GPU device on your notebook
GPU_DEVICES = tf.config.list_logical_devices('GPU')
# Get the list of all logical CPU device on your notebook
CPU_DEVICES = tf.config.list_logical_devices('CPU')
# Keep only the names of each GPU devices
GPU_DEVICES_NAMES = [x.name for x in GPU_DEVICES]
# Keep only the names of each CPU devices
CPU_DEVICES_NAMES = [x.name for x in CPU_DEVICES]
# The number of GPU devices
GPU_DEVICES_NB = len(GPU_DEVICES)
# The number of CPU devices
CPU_DEVICES_NB = len(CPU_DEVICES)

if GPU_DEVICES_NB == 0:
    raise SystemError('No GPU device found')
else:
    print(f'{GPU_DEVICES_NB} GPU device(s) have been found on your notebook :')

for nb in range(GPU_DEVICES_NB):
    gpu_name = GPU_DEVICES_NAMES[nb]
    print(f'* GPU n°{nb} whose name is "{gpu_name}"')
    
print('')
    
if CPU_DEVICES_NB == 0:
    raise SystemError('No CPU device found')
else:
    print(f'{CPU_DEVICES_NB} CPU device(s) have been found on your notebook :')

for nb in range(CPU_DEVICES_NB):
    cpu_name = CPU_DEVICES_NAMES[nb]
    print(f'* CPU n°{nb} whose name is "{cpu_name}"')

1 GPU device(s) have been found on your notebook :
* GPU n°0 whose name is "/device:GPU:0"

1 CPU device(s) have been found on your notebook :
* CPU n°0 whose name is "/device:CPU:0"


I0000 00:00:1738457308.499026    1632 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5137 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
def random_multiply(vector_length):
    vector_1 = tf.random.normal(vector_length)
    vector_2 = tf.random.normal(vector_length)
    return vector_1 * vector_2

def gpu_operation(vector_length):
    # If you have several GPU you can select the one to use by changing the used index of GPU_DEVICES_NAMES
    with tf.device(GPU_DEVICES_NAMES[0]):
        random_multiply(vector_length)
        
def cpu_operation(vector_length):
    # If you have several CPU you can select the one to use by changing the used index of GPU_DEVICES_NAMES
    with tf.device(CPU_DEVICES_NAMES[0]):
        random_multiply(vector_length)

In [5]:
import timeit

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu_operation([1])
gpu_operation([1])

for i in range(9):
    vector_length = pow(10, i)
    cpu_time = timeit.timeit(f'cpu_operation([{vector_length}])', number=20, setup="from __main__ import cpu_operation")
    gpu_time = timeit.timeit(f'gpu_operation([{vector_length}])', number=20, setup="from __main__ import gpu_operation")
    print(f'Operations on vector of length {vector_length} are {cpu_time/gpu_time}x faster on GPU than CPU')

Operations on vector of length 1 are 0.2673338796696081x faster on GPU than CPU
Operations on vector of length 10 are 0.8505639143882999x faster on GPU than CPU
Operations on vector of length 100 are 0.43602837228543667x faster on GPU than CPU
Operations on vector of length 1000 are 0.48919103232787275x faster on GPU than CPU
Operations on vector of length 10000 are 1.5784563388358728x faster on GPU than CPU
Operations on vector of length 100000 are 2.923624943511755x faster on GPU than CPU
Operations on vector of length 1000000 are 16.73305110986887x faster on GPU than CPU
Operations on vector of length 10000000 are 141.4996162494821x faster on GPU than CPU
Operations on vector of length 100000000 are 1160.2650143649378x faster on GPU than CPU


In [ ]:
# all this has really proven is that the Colab GPUs are better than yours and this was entirely a waste of time.
# i guess it got me to learn WSL and Docker?